In [1]:
import requests
import pandas as pd
import time
import urllib3
import warnings
import os
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

# UTMB

'https://api.utmb.world/races/142.hokautmbmont-blancutmb.2024/results'

In [ ]:
# Folder
output_folder = r"C:\Users\pablo.sastre-garau\Desktop\Développement\UTMB2025\UTMB2025\data\raw_results"
os.makedirs(output_folder, exist_ok=True)

# Parameters
headers = {
    "User-Agent": "Mozilla/5.0",
    "x-tenant-id": "montblanc"
}
limit = 100
gender = ''
excluded_years = [2020]

# Years declaration
for year in range(2024, 2002, -1):  # de 2024 à 2003
    if year in excluded_years:
        continue

    file_path = os.path.join(output_folder, f"utmb_results_{year}.csv")
    if os.path.exists(file_path):
        print(f"Results {year} already downloaded.")
        continue

    print(f"Results of {year} currently downloading...")
    offset = 0
    all_results = []

    base_url = f"https://api.utmb.world/races/142.hokautmbmont-blancutmb.{year}/results"

    while True:
        url = f"{base_url}?lang=fr&offset={offset}&limit={limit}&gender={gender}"
        try:
            response = requests.get(url, headers=headers, verify=False)
        except Exception as e:
            print(f"Connexion fail for {year} : {e}")
            break

        if response.status_code != 200:
            print(f"Error {response.status_code} for {year} at the offset {offset}")
            break

        data = response.json()
        runners = data.get("results", [])

        if not runners:
            break

        all_results.extend(runners)
        offset += limit
        time.sleep(0.5)

    if all_results:
        df = pd.DataFrame(all_results)
        df.to_csv(file_path, index=False)
        print(f"{len(df)} Results downloaded for {year}")
    else:
        print(f"No results {year}")

# ALL RUNNERS 

There is almost 1 200 000 runners referenced on the 'https://utmb.world/utmb-index/runner-search'

Here, let's try to get the runner_id, fullname, index, gender and category of each.

Then we'll try to have all the runners of the 2025 edition.

No mutliprocessing or asyncio for overcharge the API.

In [ ]:
# Parameters for batch processing
batch_index = 0  # 1, 2, 3, etc.
batch_size = 100_000
limit = 100
offset_start = batch_index * batch_size
offset_end = offset_start + batch_size

base_url = "https://api.utmb.world/search/runners"
output_file = fr"C:\Users\pablo.sastre-garau\Desktop\Développement\UTMB2025\UTMB2025\data\runners_{offset_start:06}_to_{offset_end-1:06}.csv"

headers = {
    "User-Agent": "Mozilla/5.0",
    "x-tenant-id": "montblanc"
}

# === SI LE FICHIER EXISTE DÉJÀ, NE PAS REFAIRE ===
if os.path.exists(output_file):
    print(f"⚠️ Fichier déjà existant : {output_file}")
else:
    print(f"🔄 Démarrage du batch {batch_index} : offsets {offset_start} à {offset_end-1}")
    all_runners = []
    seen_ids = set()
    offset = offset_start

    while offset < offset_end:
        url = f"{base_url}?category=general&sex=&ageGroup=&nationality=&limit={limit}&offset={offset}&search="
        try:
            response = requests.get(url, headers=headers, verify=False)
        except Exception as e:
            print(f"Erreur de connexion à l'offset {offset} : {e}")
            break

        if response.status_code != 200:
            print(f"❌ Erreur {response.status_code} à l'offset {offset}")
            break

        data = response.json()
        runners = data.get("runners", [])
        if not runners:
            print("✅ Fin des résultats.")
            break

        new_runners = [r for r in runners if r["id"] not in seen_ids]
        for runner in new_runners:
            seen_ids.add(runner["id"])
            all_runners.append(runner)

        offset += limit
        print(f"✔️ Runners cumulés : {len(all_runners)} (offset {offset})")
        time.sleep(0.5)

    # Enregistrement
    df = pd.DataFrame(all_runners)
    df.to_csv(output_file, index=False)
    print(f"📁 Batch {batch_index} enregistré : {output_file}")


Runners récupérés : 100
Runners récupérés : 200
Runners récupérés : 300
Runners récupérés : 400
Runners récupérés : 500
Runners récupérés : 600
Runners récupérés : 700
Runners récupérés : 800
Runners récupérés : 900
Runners récupérés : 1000
Runners récupérés : 1100
Runners récupérés : 1200
Runners récupérés : 1300
Runners récupérés : 1400
Runners récupérés : 1500
Runners récupérés : 1600
Runners récupérés : 1700
Runners récupérés : 1800
Runners récupérés : 1900
Runners récupérés : 2000
Runners récupérés : 2100
Runners récupérés : 2200
Runners récupérés : 2300
Runners récupérés : 2400
Runners récupérés : 2500
Runners récupérés : 2600
Runners récupérés : 2700
Runners récupérés : 2800
Runners récupérés : 2900
Runners récupérés : 3000
Runners récupérés : 3100
Runners récupérés : 3200
Runners récupérés : 3300
Runners récupérés : 3400
Runners récupérés : 3500
Runners récupérés : 3600
Runners récupérés : 3700
Runners récupérés : 3800
Runners récupérés : 3900
Runners récupérés : 4000
Runners r

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))

: 